In [24]:
from model import network_mnist,naive_train,test_taskwise,test,benchmark,train_stream,test_stream,compute_fisher_information,apply_importance_mask
from torch.utils.data import DataLoader, Subset
import torch
import torch.nn as nn
import torch.optim as optim


In [25]:

model=network_mnist(256,128)
#print(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs=5

acc_lists=[]
for task in range(5):
    print(f"Training on task {task}...")
    naive_train(model,task, epochs,criterion=criterion,optimizer=optimizer,device=device)
    print(f"Testing on task {task}...")
    #test_taskwise(model,task,device)
    acc,acc_list=test(model,device)
    acc_lists.append(acc_list)



Training on task 0...
Testing on task 0...
Accuracy on task 0: 99.95%
Accuracy on task 1: 0.00%
Accuracy on task 2: 0.00%
Accuracy on task 3: 0.00%
Accuracy on task 4: 0.00%
Average Accuracy: 19.99%
Training on task 1...
Testing on task 1...
Accuracy on task 0: 0.00%
Accuracy on task 1: 99.46%
Accuracy on task 2: 0.00%
Accuracy on task 3: 0.00%
Accuracy on task 4: 0.00%
Average Accuracy: 19.89%
Training on task 2...
Testing on task 2...
Accuracy on task 0: 0.00%
Accuracy on task 1: 0.00%
Accuracy on task 2: 99.68%
Accuracy on task 3: 0.00%
Accuracy on task 4: 0.00%
Average Accuracy: 19.94%
Training on task 3...
Testing on task 3...
Accuracy on task 0: 0.00%
Accuracy on task 1: 0.00%
Accuracy on task 2: 0.00%
Accuracy on task 3: 99.75%
Accuracy on task 4: 0.00%
Average Accuracy: 19.95%
Training on task 4...
Testing on task 4...
Accuracy on task 0: 0.00%
Accuracy on task 1: 0.00%
Accuracy on task 2: 0.00%
Accuracy on task 3: 0.00%
Accuracy on task 4: 98.54%
Average Accuracy: 19.71%


In [26]:
fisher_dict=compute_fisher_information(model,4,300,criterion,device)
print(fisher_dict)

{'fc1.weight': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), 'fc1.bias': tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 1.5085e-05, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        2.8277e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 2.5660e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.000

In [27]:
acc,acc_list=test(model,device)
model,mask=apply_importance_mask(model,fisher_dict,3)

print(mask)
acc1,acc_list1=test(model,device)
print("Accuracy before applying mask:",acc_list)
print("Accuracy after applying mask:",acc_list1) 

Accuracy on task 0: 0.00%
Accuracy on task 1: 0.00%
Accuracy on task 2: 0.00%
Accuracy on task 3: 0.00%
Accuracy on task 4: 98.54%
Average Accuracy: 19.71%
{'fc1.weight': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), 'fc1.bias': tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0